In [1]:
import numpy as np
import pandas as pd
import ast


In [2]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [3]:
movies = movies.merge(credits, on='title')

In [4]:
# Movie_id
# genres
# id
# keywords
# overview
# cost
# Crew

movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [5]:
movies.dropna(inplace=True)

In [6]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [7]:
def convert(obj):
    genres = []
    for genre in ast.literal_eval(obj):
        genres.append(genre['name'])
    return genres

In [8]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)

In [9]:
def convert_cast(obj):
    cast = []
    counter = 0
    for val in ast.literal_eval(obj):
        if counter != 3:
            cast.append(val['name'])
            counter += 1
        else:
            break
    return cast

In [10]:
movies['cast'] = movies['cast'].apply(convert_cast)

In [11]:
def find_director(obj):
    ret = []
    for val in ast.literal_eval(obj):
        if val['job'] == 'Director':
            ret.append(val['name'])
            break
    return ret

In [12]:
movies['crew'] = movies['crew'].apply(find_director)

In [13]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [14]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [15]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [16]:
movies.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan],"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton],"[John, Carter, is, a, war-weary,, former, mili..."


In [17]:
new_df = movies[['movie_id', 'title', 'tags']]

In [18]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

/var/folders/k6/g8rtwcms3l1dryxlhnz3jf4m0000gn/T/ipykernel_48333/3089450492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [19]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())

/var/folders/k6/g8rtwcms3l1dryxlhnz3jf4m0000gn/T/ipykernel_48333/3214958533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())


In [20]:
new_df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [21]:
import nltk

In [22]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [23]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
        
    return " ".join(y)

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [25]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [26]:
new_df['tags'] = new_df['tags'].apply(stem)

/var/folders/k6/g8rtwcms3l1dryxlhnz3jf4m0000gn/T/ipykernel_48333/3213734980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(stem)


In [27]:
from sklearn.metrics.pairwise import cosine_similarity

In [28]:
similarity = cosine_similarity(vectors)

In [29]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

[(539, 0.26089696604360174),
 (1194, 0.2581988897471611),
 (507, 0.25302403842552984),
 (260, 0.25110592822973776),
 (1216, 0.24944382578492943)]

In [38]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    
    for movie in movies_list:
        print(new_df.iloc[movie[0]])


In [39]:
recommend('Batman Begins')

movie_id                                                  155
title                                         The Dark Knight
tags        batman rais the stake in hi war on crime. with...
Name: 65, dtype: object
movie_id                                                49026
title                                   The Dark Knight Rises
tags        follow the death of district attorney harvey d...
Name: 3, dtype: object
movie_id                                                 2661
title                                                  Batman
tags        the dark knight of gotham citi begin hi war on...
Name: 1363, dtype: object
movie_id                                                  415
title                                          Batman & Robin
tags        along with crime-fight partner robin and new r...
Name: 210, dtype: object
movie_id                                                  268
title                                                  Batman
tags        the dark knight of got

In [32]:
import pickle

In [33]:
pickle.dump(new_df, open('movies.pkl','wb'))

In [35]:
new_df['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [36]:
pickle.dump(new_df.to_dict(),open('movie_dict.pkl', 'wb'))

In [37]:
pickle.dump(similarity,open('similarity.pkl','wb'))